# Importing libs and connecting to AWS

In [1]:
import connectors
import json
import pandas as pd
s3 = connectors.aws_resource()

# Reading files from S3 as json

In [2]:
path = 'loantap_cibil_reports/cibil_006308_23881792.json'

In [3]:
obj = s3.Object('unofin-prod-private', path)
body = obj.get()['Body'].read()
x = json.loads(body)

In [4]:
# Extracting Lead id from path
lead_id = path[28:]

In [5]:
# Navigating into Account section of CIBIL Report
db = x['data']['CreditReport']['Account']

# Extracting data for each account

In [6]:
# Getting Account Types
items = []
for item in db:
    try:
        items.append(item['Account_NonSummary_Segment_Fields']['AccountType'])
        get_account_types = items
        
    except KeyError:
         continue

In [7]:
# Getting Current Balance
items = []
for item in db:
    try:
        items.append(item['Account_NonSummary_Segment_Fields']['CurrentBalance'])
        get_current_balance = items
    except KeyError:
        continue

In [8]:
# Getting Sanctioned Amount
items = []
for item in db:
    try:
        items.append(item['Account_NonSummary_Segment_Fields']['HighCreditOrSanctionedAmount'])
        get_sanc_amount = items
    except KeyError:
         continue

# Converting list to a dataframe for easier use

In [9]:
get_account_types = list(map(int, get_account_types))
get_current_balance = list(map(int, get_current_balance))
get_sanc_amount = list(map(int, get_sanc_amount))

In [10]:
# Creating a dataframe
df = pd.DataFrame(list(zip(get_account_types,get_current_balance,get_sanc_amount)), 
               columns =['A/c Type','Current Balance','Sanction Amount']) 

In [11]:
# Condition for active loans
df_filtered = df[df['Current Balance'] > 0] 

# Deriving desired columns

In [12]:
total_debt = sum(df['Current Balance'])
total_loans = len(df)
total_active_loans = len(df_filtered)
total_sanc_amount = sum(df['Sanction Amount'])

In [13]:
result = df['A/c Type'].isin([1,13,17,32,34,46]).any().any()
if result:
    auto_loan = 'True'
else:
    auto_loan = 'False'

In [14]:
result = df['A/c Type'].isin([2,3]).any().any()
if result:
    housing_loan = 'True'
else:
    housing_loan = 'False'

In [15]:
pl_cl =df[df['A/c Type'].isin([5,45,6])]
no_of_pl_cl = len(pl_cl)
pl_cl_amount = sum(pl_cl['Sanction Amount'])

In [16]:
pl_cl =df_filtered[df_filtered['A/c Type'].isin([5,45,6])]
active_pl_cl = len(pl_cl)
active_pl_cl_amount = sum(pl_cl['Current Balance'])

In [17]:
cc =df[df['A/c Type'].isin([10,35,36,37])]
no_of_cc = len(cc)
cc_amount = sum(cc['Sanction Amount'])

In [18]:
cc =df_filtered[df_filtered['A/c Type'].isin([10,35,36,37])]
active_cc = len(cc)
active_cc_amount = sum(cc['Current Balance'])

# Calculating CIBIL Vintage

In [19]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
items = []
dates = []
for item in db:
    try:
        items.append(item['Account_NonSummary_Segment_Fields']['DateOpenedOrDisbursed'])
        get_vintage = items
        for s in get_vintage:
         date = datetime.strptime(s, '%d%m%Y')
         dates.append(date)
         vintage = dates
    except KeyError:
         continue

In [20]:
vin_date = min(vintage)
vintage = relativedelta(datetime.now(),vin_date).years * 12 + relativedelta(datetime.now(),vin_date).months

# Creating final dataframe for use

In [21]:
data = pd.DataFrame({
    'id': lead_id,
    'total_loans': total_loans,
    'total_active_loans' : total_active_loans,
    'total_sanc_amount' : total_sanc_amount,
    'total_debt': total_debt,
    'auto_loan_flag': auto_loan,
    'housing_loan_flag': housing_loan,
    'no_of_relevant_loans': no_of_pl_cl,
    'relevant_loans_sanc_amount' : pl_cl_amount,
    'active_relevant_loans' : active_pl_cl,
    'current_balance_relevant_loans' : active_pl_cl_amount,
    'no_of_credit_cards': no_of_cc,
    'credit_card_sanc_amount': cc_amount,
    'active_credit_cards': active_cc,
    'current_balance_credit_cards': active_cc_amount,
    'cibil vintage' : vintage
}, index=[0])

In [22]:
data.head()

,id,total_loans,total_active_loans,total_sanc_amount,total_debt,auto_loan_flag,housing_loan_flag,no_of_relevant_loans,relevant_loans_sanc_amount,active_relevant_loans,current_balance_relevant_loans,no_of_credit_cards,credit_card_sanc_amount,active_credit_cards,current_balance_credit_cards,cibil vintage
0,006308_23881792.json,3,3,840780,601252,True,False,0,0,0,0,1,37780,1,16306,46
